<a href="https://colab.research.google.com/github/HaeunJeon/Jeju_Rec/blob/main/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **파일불러오기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
def bring():
  df = pd.read_excel("/content/drive/MyDrive/종합설계/final_review2.xlsx")
  df.to_csv
  df=df.loc[:,['place', 'review']]
  return df

# **군집화**

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
def clustering(content):
  # 군집화 할 그룹의 갯수 정의
  n_clusters = len(content)//50

    # CountVectrizer로 토큰화
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(content)

    # l2 정규화
  X = normalize(X)

    # k-means 알고리즘 적용
  kmeans = KMeans(n_clusters=n_clusters).fit(X)

    # trained labels and cluster centers
  labels = kmeans.labels_
  centers = kmeans.cluster_centers_

    # labels에 merge
  place['labels'] = labels
  df = place
  return kmeans

# **빈도수**

In [5]:
#빈도수 계산 위한 tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# ============================================
# -- Get TFIDF
# ============================================
def tfidf(content):
  vectorizer = TfidfVectorizer()
  sp_matrix = vectorizer.fit_transform(content)

  word2id = defaultdict(lambda : 0)
  for idx, feature in enumerate(vectorizer.get_feature_names()):
      word2id[feature] = idx

  for i, sent in enumerate(content):
      print('====== document[%d] ======' % i)
      print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
  return vectorizer

# **상위 feature 출력**

In [6]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    # 각 클러스터 레이블별 feature들의 center값들 내림차순으로 정렬 후의 인덱스를 반환
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 클러스터 레이블별로 
    for cluster_num in range(cluster_num):
        # 개별 클러스터별 정보를 담을 empty dict할당
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # 각 feature별 center값들 정렬한 인덱스 중 상위 20개만 추출
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        # top_ftr_idx를 활용해서 상위 20개 feature들의 center값들 반환
        # 반환하게 되면 array이기 떄문에 리스트로바꾸기
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        # cluster_details 딕셔너리에다가 개별 군집 정보 넣어주기
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print()
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print('-'*20)

# **장소입력 및 실행**

In [34]:
#장소입력
df = bring()
where=df.loc[:,['place']]
where.drop_duplicates(subset='place',inplace=True,ignore_index=True) #장소 중복제거 
where = where['place'].tolist()

In [39]:
place = df.loc[df.place == where[0]]
content= place['review'].apply(lambda x : str(x))
kmeans = clustering(content)
vectorizer = tfidf(content)
feature_names = vectorizer.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans,
                                    cluster_data=content,
                                    feature_names=feature_names,
                                    cluster_num=len(content)//50,
                                    top_n_features=20)
print_cluster_details(cluster_details)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


====== document[0] ======
[('얼마', 0.1934547876069097), ('전', 0.0), ('가족들과', 0.2041324759749243), ('외식도', 0.24490887735334374), ('할', 0.0), ('겸', 0.0), ('도민들이', 0.2298595208481413), ('많이', 0.1469568864253041), ('방문한다는', 0.24490887735334374), ('곽지해수욕장', 0.12063848471530322), ('맛', 0.0), ('집에서', 0.18908311946972187), ('식사를', 0.21089957393701028), ('했다', 0.1984109761717381), ('맛과', 0.24490887735334374), ('가게', 0.18517252906379328), ('안으로', 0.2191818324801267), ('들어가니', 0.2298595208481413), ('은은한', 0.24490887735334374), ('조명과', 0.24490887735334374), ('원목', 0.24490887735334374), ('인테리어가', 0.2298595208481413), ('돋보이는', 0.24490887735334374), ('곽지해수욕장', 0.12063848471530322), ('맛', 0.0), ('집의', 0.21089957393701028), ('홀과', 0.24490887735334374)]
====== document[1] ======
[('헷', 0.0), ('코스', 0.2111002638489112), ('카페', 0.14684770094474459), ('청춘', 0.2532685129169862), ('부부', 0.2266633099239258), ('신창', 0.2532685129169862), ('풍차', 0.2532685129169862), ('해안도로', 0.2000581069308654), ('곽지해수욕장', 0.0623